# Please visit : https://blog.naver.com/sorryhyun96/223138364761 for more detail & funny pepe

In [ ]:
!pip install transformers datasets evaluate sentencepiece
!pip install accelerate -U

In [ ]:
from datasets import load_dataset
sst2_dataset = load_dataset("glue",'sst2')
sst2_dataset['train'][:5]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'sentence': ['hide new secretions from the parental units ',
  'contains no wit , only labored gags ',
  'that loves its characters and communicates something rather beautiful about human nature ',
  'remains utterly satisfied to remain the same throughout ',
  'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up '],
 'label': [0, 0, 1, 0, 0],
 'idx': [0, 1, 2, 3, 4]}

In [ ]:
from transformers import TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, set_seed
import numpy as np
import evaluate

def make_trainer(save_dir, model_path, dataset, coulmn_names):
  set_seed(42)
  model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
  tokenizer = AutoTokenizer.from_pretrained(model_path)

  def tokenize_function(examples):
      return tokenizer(examples[coulmn_names["sentence_column"]], truncation=True, max_length=64)

  tokenized_train = dataset[coulmn_names['train_column']].map(tokenize_function, batched=True)
  tokenized_valid = dataset[coulmn_names['val_column']].map(tokenize_function, batched=True)

  collator = DataCollatorWithPadding(tokenizer=tokenizer)

  metric = evaluate.load("accuracy")
  def compute_metrics(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)

  from transformers import TrainingArguments, Trainer
  training_args = TrainingArguments(output_dir=save_dir, eval_steps=500, max_steps=2500, learning_rate=2e-5,
                                    metric_for_best_model='eval_accuracy', evaluation_strategy='steps',
                                    load_best_model_at_end=True, save_total_limit=2, per_device_train_batch_size=24,
                                    dataloader_num_workers=2)
  return Trainer(
      model=model,
      tokenizer=tokenizer,
      args=training_args,
      data_collator=collator,
      train_dataset=tokenized_train,
      eval_dataset=tokenized_valid,
      compute_metrics=compute_metrics)


k-roberta --Finetune--> sst2 (83.94) --Evaluate--> nsmc (68.66)

In [ ]:
sst2_coulmn_names = {'sentence_column':'sentence', 'train_column':'train', 'val_column':'validation'}
K_roberta_path="klue/roberta-base"
klue_roberta_sst2_trainer = make_trainer('klue_roberta_sst2', K_roberta_path, sst2_dataset, sst2_coulmn_names)
klue_roberta_sst2_trainer.train()
klue_roberta_sst2_finetuned_model_path = klue_roberta_sst2_trainer.state.best_model_checkpoint

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.557800,0.472883,0.795872
1000,0.427900,0.436403,0.792431
1500,0.369100,0.436955,0.824541
2000,0.330000,0.381785,0.839450
2500,0.313600,0.383028,0.833716


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

In [ ]:
nsmc_dataset = load_dataset("nsmc")
nsmc_dataset['train'][:5]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'id': ['9976970', '3819312', '10265843', '9045019', '6483659'],
 'document': ['아 더빙.. 진짜 짜증나네요 목소리',
  '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  '너무재밓었다그래서보는것을추천한다',
  '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'],
 'label': [0, 1, 0, 0, 1]}

In [ ]:
nsmc_coulmn_names = {'sentence_column':'document', 'train_column':'train', 'val_column':'test'}
klue_roberta_sst2_nsmc_trainer = make_trainer('temp', klue_roberta_sst2_finetuned_model_path, nsmc_dataset, nsmc_coulmn_names)
klue_roberta_sst2_nsmc_trainer.evaluate()

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.5722680687904358,
 'eval_accuracy': 0.68666,
 'eval_runtime': 154.2275,
 'eval_samples_per_second': 324.196,
 'eval_steps_per_second': 40.525}

xlm_roberta --Finetune--> sst2 (91.51) --Evaluate--> nsmc (78.86)

In [ ]:
multilingual_model_path = "xlm-roberta-base"
xlm_roberta_sst2_trainer = make_trainer('xlm_roberta_sst2', multilingual_model_path, sst2_dataset, sst2_coulmn_names)
xlm_roberta_sst2_trainer.train()
xlm_roberta_sst2_finetuned_model_path = xlm_roberta_sst2_trainer.state.best_model_checkpoint

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.d

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.452300,0.281397,0.896789
1000,0.308700,0.251076,0.907110
1500,0.267000,0.251345,0.903670
2000,0.233000,0.232650,0.912844
2500,0.221100,0.250957,0.915138


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with 

In [ ]:
xlm_roberta_sst2_nsmc_trainer = make_trainer('temp',xlm_roberta_sst2_finetuned_model_path, nsmc_dataset, nsmc_coulmn_names)
xlm_roberta_sst2_nsmc_trainer.evaluate()

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.5969642996788025,
 'eval_accuracy': 0.78862,
 'eval_runtime': 156.7848,
 'eval_samples_per_second': 318.908,
 'eval_steps_per_second': 39.864}

k-roberta --Finetune--> nsmc (89.65) --Evaluate--> sst2 (60.21)

In [ ]:
klue_roberta_nsmc_trainer = make_trainer('klue_roberta_nsmc', K_roberta_path, nsmc_dataset, nsmc_coulmn_names)
klue_roberta_nsmc_trainer.train()
klue_roberta_nsmc_finetuned_model_path = klue_roberta_nsmc_trainer.state.best_model_checkpoint

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.370400,0.306282,0.866120
1000,0.301400,0.325141,0.881180
1500,0.270300,0.273163,0.889960
2000,0.276100,0.259906,0.895960
2500,0.257100,0.256229,0.896480


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

In [ ]:
klue_roberta_nsmc_sst2_trainer = make_trainer('temp', klue_roberta_nsmc_finetuned_model_path, sst2_dataset, sst2_coulmn_names)
klue_roberta_nsmc_sst2_trainer.evaluate()

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6700038909912109,
 'eval_accuracy': 0.6020642201834863,
 'eval_runtime': 3.4349,
 'eval_samples_per_second': 253.862,
 'eval_steps_per_second': 31.733}

xlm_roberta --Finetune--> nsmc (87.44) --Evaluate--> sst2 (84.29)

In [ ]:
xlm_roberta_nsmc_trainer = make_trainer('xlm_roberta_nsmc', multilingual_model_path, nsmc_dataset, nsmc_coulmn_names)
xlm_roberta_nsmc_trainer.train()
xlm_roberta_nsmc_finetuned_model_path = xlm_roberta_nsmc_trainer.state.best_model_checkpoint

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.d

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.502200,0.398760,0.835520
1000,0.369400,0.397600,0.842600
1500,0.337500,0.312236,0.866560
2000,0.329800,0.307415,0.873240
2500,0.305800,0.303959,0.874460


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with 

In [ ]:
xlm_roberta_nsmc_sst2_trainer = make_trainer('temp',xlm_roberta_nsmc_finetuned_model_path, sst2_dataset, sst2_coulmn_names)
xlm_roberta_nsmc_sst2_trainer.evaluate()

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.3640556037425995,
 'eval_accuracy': 0.8428899082568807,
 'eval_runtime': 3.0428,
 'eval_samples_per_second': 286.574,
 'eval_steps_per_second': 35.822}

mdeberta_v3 --Finetune--> nsmc (88.79) --Evaluate--> sst2 (87.73)

In [ ]:
sst2_coulmn_names = {'sentence_column':'sentence', 'train_column':'train', 'val_column':'validation'}
nsmc_coulmn_names = {'sentence_column':'document', 'train_column':'train', 'val_column':'test'}

In [ ]:
better_multilingual_model_path = "microsoft/mdeberta-v3-base"
mdeberta_nsmc_trainer = make_trainer('mdeberta_nsmc', better_multilingual_model_path, nsmc_dataset, nsmc_coulmn_names)
mdeberta_nsmc_trainer.train()
mdeberta_nsmc_finetuned_model_path = mdeberta_nsmc_trainer.state.best_model_checkpoint

Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'deberta.embeddings.word_embeddings._weight', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializi

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.446500,0.350485,0.848400
1000,0.339900,0.310138,0.876080
1500,0.305900,0.292701,0.881060
2000,0.301700,0.278878,0.884020
2500,0.277900,0.283484,0.887940


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fas

In [ ]:
mdeberta_nsmc_sst2_trainer = make_trainer('temp', mdeberta_nsmc_finetuned_model_path, sst2_dataset, sst2_coulmn_names)
mdeberta_nsmc_sst2_trainer.evaluate()

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.31072160601615906,
 'eval_accuracy': 0.8772935779816514,
 'eval_runtime': 4.4775,
 'eval_samples_per_second': 194.752,
 'eval_steps_per_second': 24.344}

mdeberta_v3 --Finetune--> sst2 () --Evaluate--> nsmc ()

In [ ]:
mdeberta_sst2_trainer = make_trainer('mdeberta_sst2', better_multilingual_model_path, sst2_dataset, sst2_coulmn_names)
mdeberta_sst2_trainer.train()
mdeberta_sst2_finetuned_model_path = mdeberta_sst2_trainer.state.best_model_checkpoint

Some weights of the model checkpoint at microsoft/mdeberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'deberta.embeddings.word_embeddings._weight', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializi

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.347900,0.223619,0.910550
1000,0.232000,0.255021,0.920872
1500,0.210800,0.271565,0.927752
2000,0.191200,0.228253,0.924312
2500,0.179700,0.252334,0.925459


You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fas

In [ ]:
mdeberta_sst2_nsmc_trainer = make_trainer('temp',mdeberta_sst2_finetuned_model_path, nsmc_dataset, nsmc_coulmn_names)
mdeberta_sst2_nsmc_trainer.evaluate()

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6829042434692383,
 'eval_accuracy': 0.80272,
 'eval_runtime': 251.0968,
 'eval_samples_per_second': 199.126,
 'eval_steps_per_second': 24.891}